<a href="https://colab.research.google.com/github/prashanth018/Text-Summarization/blob/Dyuthi/Copy_of_Text_summarization_using_word_frequencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import bs4 as bs
import urllib.request
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install rouge_metric

     |████████████████████████████████| 153kB 5.6MB/s 


In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149815 sha256=5316111e3204f63d2588feec0c19b42860d5292f73159db3f154b92bb5476edc
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving news_category_dataset-161000-171000.csv to news_category_dataset-161000-171000.csv


In [ ]:
dataset = pd.read_csv('news_category_dataset-161000-171000.csv')
news_text = dataset['news_text']
news_text

0       This undated publicity film image provided by ...
1       WASHINGTON, UNITED STATES: The Easter Bunny wa...
2       1 / 712 Animal Photos Of The Week (9/16-9/22)\...
3       * Melted ice re-freezes faster than sea water ...
4       Co-written by Pushker Kharecha and Makiko Sato...
                              ...                        
9995    Last week I shared how I was able to get off c...
9996    "It is not true that money can't buy happiness...
9997    Even if you don't, there's no need to be afrai...
9998    After stumbling across this peculiar Check You...
9999    Officials in Carson City, Nev., shocked a Cali...
Name: news_text, Length: 10000, dtype: object

In [ ]:
def casefolding(sentence):
    return sentence.lower()

def cleaning(sentence):
    return re.sub(r'[^a-z]', ' ', re.sub("’", '', sentence))

def tokenization(sentence):
    return sentence.split()

In [ ]:
def sentence_split(paragraph):
    return nltk.sent_tokenize(paragraph)

In [ ]:
def word_freq(data):
    w = []
    for sentence in data:
        for words in sentence:
            w.append(words)
    bag = list(set(w))
    res = {}
    for word in bag:
        res[word] = w.count(word)
    return res

def sentence_weight(data):
    weights = []
    for words in data:
        temp = 0
        for word in words:
            temp += wordfreq[word]
        weights.append(temp)
    return weights

In [ ]:
for news in range(len(news_text)):
    #print(lst[news])
    #print(news_text[news])
    sentence_list = sentence_split(str(news_text[news]))
    data = []
    for sentence in sentence_list:
        data.append(tokenization(cleaning(casefolding(sentence))))
    data = (list(filter(None, data)))
    wordfreq = word_freq(data)
    ranking = []
    for words in data:
        temp = 0
        for word in words:
            temp += wordfreq[word]
        ranking.append(temp)
    n = 1
    result = ''
    sort_list = np.argsort(ranking)[::-1][:n]
    #print(sort_list)
    l = []
    for i in range(n):
        result += '{} '.format(sentence_list[sort_list[i]])
    print(result)

In [ ]:
news = '''
O'Sullivan could run in Worlds
Sonia O'Sullivan has indicated that she would like to participate in next month's World Cross Country Championships in St Etienne.
Athletics Ireland have hinted that the 35-year-old Cobh runner may be included in the official line-up for the event in France on 19-20 March. 
Provincial teams were selected after last Saturday's Nationals in Santry and will be officially announced this week. 
O'Sullivan is at present preparing for the London marathon on 17 April. 
The participation of O'Sullivan, currentily training at her base in Australia, would boost the Ireland team who won the bronze three years agio. 
The first three at Santry last Saturday, Jolene Byrne, Maria McCambridge and Fionnualla Britton, are automatic selections and will most likely form part of the long-course team. 
O'Sullivan will also take part in the Bupa Great Ireland Run on 9 April in Dublin. 
'''

In [ ]:
sentence_list = sentence_split(str(news))
data = []
for sentence in sentence_list:
    data.append(tokenization(cleaning(casefolding(sentence))))
data = (list(filter(None, data)))
wordfreq = word_freq(data)
ranking = []
for words in data:
    temp = 0
    for word in words:
        temp += wordfreq[word]
    ranking.append(temp)
n = 2
result = ''
sort_list = np.argsort(ranking)[::-1][:n]
#print(sort_list)
for i in range(n):
    result += '{} '.format(sentence_list[sort_list[i]])
print(result)

The participation of O'Sullivan, currentily training at her base in Australia, would boost the Ireland team who won the bronze three years agio. Athletics Ireland have hinted that the 35-year-old Cobh runner may be included in the official line-up for the event in France on 19-20 March. 


In [ ]:
hypotheses = list()
hypotheses.append(result.split())

In [ ]:
actual = '''
The participation of O'Sullivan, currentily training at her base in Australia, would boost the Ireland team who won the bronze three years agio.
O'Sullivan will also take part in the Bupa Great Ireland Run on 9 April in Dublin.
O'Sullivan is at present preparing for the London marathon on 17 April.
'''

In [ ]:
references = list()
references.append(actual.split())

In [ ]:
from rouge_metric import PyRouge
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate_tokenized(hypotheses, references)
print(scores)

{'rouge-1': {'r': 0.9718875502008032, 'p': 0.019689203482222764, 'f': 0.03859649122807018}, 'rouge-2': {'r': 0.8585858585858586, 'p': 0.013888888888888888, 'f': 0.027335584499115613}, 'rouge-4': {'r': 0.7037037037037037, 'p': 0.006261328060636019, 'f': 0.012412216233872285}, 'rouge-l': {'r': 0.9718875502008032, 'p': 0.019689203482222764, 'f': 0.03859649122807018}, 'rouge-w-1.2': {'r': 0.7270575698607228, 'p': 0.01518870040037385, 'f': 0.029755783342781447}, 'rouge-s4': {'r': 0.9182282793867121, 'p': 0.008881199538638985, 'f': 0.01759224505116112}, 'rouge-su4': {'r': 0.9312101910828026, 'p': 0.010023310023310023, 'f': 0.019833141151733024}}


In [ ]:
from fuzzywuzzy import fuzz
fuzz_ratio = fuzz.ratio(result, actual)
fuzz_ratio

71